In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.io as pio
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px


In [2]:
data_anly_1= pd.read_csv('ch5_data/analyse_all_dates_10%_1h/abs.csv') 

data_anly_2= pd.read_csv('ch5_data/analyse_all_dates_10%_1h/range_width_1.csv') 

data_anly_3= pd.read_csv('ch5_data/analyse_all_dates_10%_1h/range_width_30.csv') 

data_anly_4= pd.read_csv('ch5_data/analyse_all_dates_10%_1h/range_with_halfE.csv') 


In [3]:
data_anly_1['Datetime'] = pd.to_datetime(data_anly_1['Datetime'])
data_anly_1 = data_anly_1[:len(data_anly_1)-10]
data_anly_1

,Unnamed: 0,Datetime,P_W_list,p_bss_values,E,P_W_shift,original_flucuation,P_CW,P_CW_shift,after_flucuation
0,0,2022-01-01 00:00:00,1803.48,0.00,112.72,1803.48,0.00,1803.48,1803.48,0.00
1,1,2022-01-01 00:15:00,1962.39,-160.76,150.18,1803.48,158.91,1801.63,1803.48,-1.85
2,2,2022-01-01 00:30:00,1801.63,0.00,150.18,1962.39,-160.76,1801.63,1801.63,0.00
3,3,2022-01-01 00:45:00,1824.41,-218.73,201.14,1801.63,22.78,1605.68,1801.63,-195.95
4,4,2022-01-01 01:00:00,1661.57,-55.91,214.17,1824.41,-162.84,1605.66,1605.68,-0.02
...,...,...,...,...,...,...,...,...,...,...
34995,34995,2022-12-31 12:45:00,2184.38,-22.48,129.77,2182.35,2.03,2161.90,2161.90,0.00
34996,34996,2022-12-31 13:00:00,2178.86,-16.70,133.66,2184.38,-5.52,2162.16,2161.90,0.26
34997,34997,2022-12-31 13:15:00,2189.13,-26.78,139.90,2178.86,10.27,2162.35,2162.16,0.19
34998,34998,2022-12-31 13:30:00,2168.09,-5.56,141.19,2189.13,-21.04,2162.53,2162.35,0.18


- 仿真程序流程
- 		画图
- 		仿真结果和分析
- 	仿真的结果包含
- 		4个方案的全年运行结果
- 			只需要包含功率列和能量列
- 	仿真的结果分析包含
- 		4个方案的全年考核量
- 		4个方案的连续向上波动、连续向下波动、一上一下波动的优化统计（观察对于典型场景的总体应对效果）
- 			原先连续波动“长度-波动总值”分布
- 仿真的可视化
- 		每个方案验证了前面猜想的图（需要观察数据进行画图）
- 		日尺度图
- 		月尺度图（说明选择该图的原因（比如每种波动向上波动向下波动频繁上下波动长波动均包含））
- 4个方案的典型日图（可采取同一天，用两天）（体现不同方案的应对）
- 		4个方案的典型月图（可整体观察优化效果，而非局限于日）


# 可视化函数

In [4]:
def visualization_interactive(data_season_test, upper_baseline, lower_baseline,start,end):# ,route
    # 函数功能：可视化
    # 函数输入：数据集，上下准线
    data_season = data_season_test.copy()[start:end]

    # 使用Plotly创建交互式图表
    fig = make_subplots(rows=1, cols=1) # 实际上只画了一个图

    # 添加数据线
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['original_flucuation'], mode='lines+markers', name='波动值'))
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['after_flucuation'], mode='lines+markers', name='后波动值'))
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['p_bss_values'], mode='lines+markers', name='储能功率值'))
    fig.add_trace(go.Scatter(x=data_season['Datetime'], y=data_season['E'], mode='lines+markers', name='储能能量值'))

    # 添加基线
    fig.add_hline(y=upper_baseline, line_dash="dash", line_color="orange", annotation_text="上波动考核线", annotation_position="bottom right")
    fig.add_hline(y=lower_baseline, line_dash="dash", line_color="red", annotation_text="下波动考核线", annotation_position="top right")

    # 设置图表布局
    fig.update_layout(
        title='波动情况分析',
        xaxis_title='日期时间',
        yaxis_title='波动值 (MW)',
        legend_title='图例',
        font=dict(family="SimHei", size=12)
    )
    
    # 设置X轴为日期时间格式
    fig.update_xaxes(tickformat="%Y-%m-%d %H:%M:%S")
    # fig.savefig('ch5fig/range_test.png')
    # 显示图表
    # fig.write_image(route)
    fig.show()


In [5]:

# 分别绘制并输出4张图片，每张图片显示一个季节的波动情况和基线
data_season_test = data_anly_1.copy()
visual_length = 96

# 考核标准的衡量
upper_baseline = 50  # MW
lower_baseline = -50  # MW

# 使用修改后的函数绘制交互式图像
visualization_interactive(data_season_test, upper_baseline, lower_baseline,0,visual_length)#,fig_route


In [6]:
# 分别绘制并输出4张图片，每张图片显示一个季节的波动情况和基线
data_season_test = data_anly_2.copy()
visual_length = 96

# 考核标准的衡量
upper_baseline = 50  # MW
lower_baseline = -50  # MW

# 使用修改后的函数绘制交互式图像
visualization_interactive(data_season_test, upper_baseline, lower_baseline,0,visual_length)#,fig_route


# 考核电量统计定义

In [7]:
def refine(df,folder_name): 
    # 函数功能：统计考核量（需要写一段代码算出考核量
    # 函数输入：全年数据窗口
    # 函数输出：统计结果dataframe，考核量列表（包含减少的考核电量、原考核电量、后考核电量）
    df_result = df.copy()
    df_result['mean_original_flucuation'] = df_result['abs_original_flucuation'].rolling(window=10,min_periods=1).mean()#.shift(-window_size+1)
    df_result['original_refine_electricity'] = df_result['mean_original_flucuation'][df_result['mean_original_flucuation']>50] - 50
    original_refine_amount = df_result['original_refine_electricity'].sum()*10*15/60 # 单位MW.h
    df_result['mean_after_flucuation'] = df_result['abs_after_flucuation'].rolling(window=10,min_periods=1).mean()#.shift(-window_size+1)
    df_result['after_refine_electricity'] = df_result['mean_after_flucuation'][df_result['mean_after_flucuation']>50] - 50
    after_refine_amount = df_result['after_refine_electricity'].sum()*10*15/60 # 单位MW.h
    filename = 'refine.csv'
    file_path = os.path.join(folder_name, filename)
    reduced_refine_amount = original_refine_amount - after_refine_amount
    df_result.to_csv(file_path,index=False)
    refine_list = [reduced_refine_amount,original_refine_amount,after_refine_amount]
    return df_result,refine_list

# 考核点统计定义

In [8]:
def refine_num(df): # 统计考核点的数量（）# 没用的指标！
    # 输入 目标分析数据集
    # 输出 列表包含
    # 原受考核点数量、原向上波动考核点数量、原向下波动考核点数量、优化后受考核点数量、优化后向上波动考核点数量、优化后向下波动考核点数量、
    # 原首考核点比例、原向上波动考核点波动、原向下波动考核点比例、优化后同理

    df_result = df.copy()
    original_refine_points = df_result['abs_original_flucuation'][df_result['abs_original_flucuation'] > upper_baseline].count()
    original_upper_refine_points = df_result['original_flucuation'][df_result['original_flucuation'] > upper_baseline].count()
    original_lower_refine_points = df_result['original_flucuation'][df_result['original_flucuation'] < lower_baseline].count()
    
    after_refine_points = df_result['abs_after_flucuation'][df_result['abs_after_flucuation'] > upper_baseline].count()
    after_upper_refine_points = df_result['after_flucuation'][df_result['after_flucuation'] > upper_baseline].count()
    after_lower_refine_points = df_result['after_flucuation'][df_result['after_flucuation'] < lower_baseline].count()

    original_abs_ratio = original_refine_points/len(df_result)
    original_upper_ratio = original_upper_refine_points/len(df_result)
    original_lower_ratio = original_lower_refine_points/len(df_result)

    after_abs_ratio = after_refine_points/len(df_result)
    after_upper_ratio = after_upper_refine_points/len(df_result)
    after_lower_ratio = after_lower_refine_points/len(df_result)

    refine_num_list = [original_refine_points,
                       original_upper_refine_points,
                       original_lower_refine_points,
                       after_refine_points,
                       after_upper_refine_points,
                       after_lower_refine_points,
                       original_abs_ratio,
                       original_upper_ratio,
                       original_lower_ratio,
                       after_abs_ratio,
                       after_upper_ratio,
                       after_lower_ratio]
    return refine_num_list


# 连续波动特点统计定义

In [9]:
def continuou_fluctuation_analysis(df,mode,time_mode):
    # 功能描述：分析连续波动的各项指标
    # 函数输入：原始数据集，模式，保存文件夹
    # 函数输出：分析数据集
    df_positive_copy = df.copy()

    if time_mode == 'original':
        df_positive_copy['Fluctuation15M'] = df_positive_copy['original_flucuation']
    elif time_mode == 'after':
        df_positive_copy['Fluctuation15M'] = df_positive_copy['after_flucuation']


    if mode == 'positive':
        df_positive_copy['is_positive'] = df_positive_copy['Fluctuation15M'] > 0
    elif mode == 'negative':
        df_positive_copy['is_positive'] = df_positive_copy['Fluctuation15M'] < 0
        df_positive_copy['is_positive'] = -df_positive_copy['is_positive']
    
    # 进行绝对值的处理
    df_positive_copy['is_positive'] = abs(df_positive_copy['is_positive'])
    
    # 检测连续正数序列的开始
    df_positive_copy['new_group'] = df_positive_copy['is_positive'] & (df_positive_copy['is_positive'] != df_positive_copy['is_positive'].shift(1))
    # 为每个连续正数序列分配一个唯一的标识
    df_positive_copy['group_id'] = df_positive_copy['new_group'].cumsum()
    # 仅选取正数进行分组和计数
    positive_sequences = df_positive_copy[df_positive_copy['is_positive']].groupby('group_id').size()
    # 首先对每个连续正数序列进行聚合计算，得到波动总值、平均值和最大值
    grouped_stats = df_positive_copy[df_positive_copy['is_positive']].groupby('group_id')['Fluctuation15M'].agg(
        total_fluctuation='sum',
        average_fluctuation='mean',
        max_fluctuation='max'
    ).reset_index()

    # 将原先计算的序列长度也加入统计表中
    grouped_stats['sequence_length'] = grouped_stats['group_id'].map(positive_sequences)

    # 现在grouped_stats包含了每个连续正数序列的长度、波动总值、平均值和最大值
    # 对这些统计值进行进一步的汇总统计
    summary_stats = grouped_stats.groupby('sequence_length').agg({
        'total_fluctuation': ['sum', 'mean', 'max'],
        'average_fluctuation': ['mean'],
        'max_fluctuation': ['max']
    })
    # 计算各长度连续波动的数量
    sequence_counts = positive_sequences.groupby(positive_sequences).size().reset_index()
    sequence_counts.columns = ['sequence_length', 'count']

    # 合并各项统计数据
    summary_stats = pd.merge(summary_stats, sequence_counts, on='sequence_length', how='left')

    # 重命名列以便更清晰地表示统计结果
    summary_stats.columns = ['sequence_length', 'total_fluctuation_sum', 'total_fluctuation_mean', 'total_fluctuation_max',
                             'average_fluctuation_mean', 'max_fluctuation_max', 'count']
    # # 重命名列以便更清晰地表示统计结果
    # summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]
    # summary_stats = summary_stats.reset_index()

    return summary_stats



# 对上下波动分开统计

In [10]:
def classic_fluctuation_stata(df,folder) :# 统计典型波动优化效果
    # 函数功能：经典波动量数据
    # 函数输入：数据集、上波动分析结果路径、下波动分析结果路径
    # 函数输出：总结列表、向上向下波动统计列表（包含原上波动功率值求和、原向下波动求和功率、优化后上波动功率值求和、优化后向下波动求和功率）

    df_result = df.copy()
    # 后上下波动统计
    original_upper_amount = df_result['original_flucuation'][df_result['original_flucuation'] > upper_baseline].sum()
    original_lower_amount = df_result['original_flucuation'][df_result['original_flucuation'] < lower_baseline].sum()
    # 后上下波动统计
    after_upper_amount = df_result['after_flucuation'][df_result['after_flucuation'] > upper_baseline].sum()
    after_lower_amount = df_result['after_flucuation'][df_result['after_flucuation'] < lower_baseline].sum()

    summary_stats_list = []
    for i in ('positive','negative'):
        for j in ('original','after'):
            file_name = i+'_'+j+'_'+'summary.csv'
            stats = continuou_fluctuation_analysis(df_result,i,j)
            # 拼接文件夹路径和文件名
            file_path = os.path.join(folder, file_name)
            # 将结果保存到CSV文件
            stats.to_csv(file_path,index=False)
            summary_stats_list.append(stats)
    
    classic_fluctuation_list = [original_upper_amount,original_lower_amount,after_upper_amount,after_lower_amount]
    return summary_stats_list,classic_fluctuation_list


# 对数据绝对值处理定义

In [11]:
def absolute_data(df):
    df_abs = df.copy()
    df_abs['abs_original_flucuation'] = abs(df['original_flucuation'])
    df_abs['abs_after_flucuation'] = abs(df['after_flucuation'])
    return df_abs

data_abs_1 = absolute_data(data_anly_1)

In [12]:
# method_1_refine_num_list = refine_num(data_abs_1)
# method_1_refine_num_list
# file = 'ch5_analysis'
# data,method_1_confluctuation_list = classic_fluctuation_stata(data_abs_1,file)


In [13]:
def statistics(data,refine_folder,stata_folder):
# 功能：对波动量进行统计 计算考核电量和各重要统计量
# 输入 储能功率数据集
# 输出 
# 波动量统计数据集，
#       原连续向上波动统计表
#       原连续向下波动统计表
#       后连续向上波动统计表
#       后连续向下波动统计表
# 各重要统计量结论列表
# 如下
# 考核量列表
#   （包含减少的考核电量、
#              原考核电量、
#              后考核电量）
# 考核点统计
#              原受考核点数量、
#              原向上波动考核点数量、
#              原向下波动考核点数量、
#              优化后受考核点数量、
#              优化后向上波动考核点数量、
#              优化后向下波动考核点数量、
# 考核点比例统计
#               原受考核点比例、
#               原向上波动考核点比例、
#               原向下波动考核点比例、
#               优化后受考核点比例、
#               优化后向上波动考核点比例、
#               优化后向下波动考核点比例、
# 向上向下波动量统计
#              原向上波动考核功率求和、
#              原向上波动考核功率求和、
#              后向上波动考核功率求和、
#              后向上波动考核功率求和

    df_pre = data.copy()
    df = absolute_data(df_pre)
    df_result,refine_list = refine(df,refine_folder)# 统计考核量（需要写一段代码算出考核量）
    refine_num_list = refine_num(df)# 统计考核点的数量（）
    summary_stats_list,classic_fluctuation_list = classic_fluctuation_stata(df,stata_folder) # 统计典型波动优化效果
    
    anly_list = refine_list+refine_num_list+classic_fluctuation_list
    stata_list = [df_result,summary_stats_list]
    return stata_list,anly_list

In [14]:
print_list = ['减少的考核电量',
              '原考核电量',
              '后考核电量',
               '原受考核点数量',
               '原向上波动考核点数量',
               '原向下波动考核点数量',
               '优化后受考核点数量',
               '优化后向上波动考核点数量',
               '优化后向下波动考核点数量',
               '原受考核点比例',
               '原向上波动考核点比例',
               '原向下波动考核点比例',
               '优化后受考核点比例',
               '优化后向上波动考核点比例',
               '优化后向下波动考核点比例',
               '原向上波动考核功率求和',
               '原向上波动考核功率求和',
               '后向上波动考核功率求和',
               '后向上波动考核功率求和']


# 调用示例

## 方案1结果分析

In [15]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates_10%_1h/abs'
stata_folder = 'ch5_data/analyse_results_of_dates_10%_1h/abs'

stata_list_1,anly_list_1 = statistics(data_anly_1,refine_foler,stata_folder)
anly_list_1

for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_1[i]}')



减少的考核电量是836948.8203373008
原考核电量是1107604.9910515868
后考核电量是270656.17071428604
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是3514
优化后向上波动考核点数量是1713
优化后向下波动考核点数量是1801
原受考核点比例是0.2543714285714286
原向上波动考核点比例是0.1276
原向下波动考核点比例是0.12677142857142856
优化后受考核点比例是0.1004
优化后向上波动考核点比例是0.04894285714285714
优化后向下波动考核点比例是0.051457142857142854
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是238625.47999999998
后向上波动考核功率求和是-223022.01


In [16]:

visualization_interactive(data_anly_1, upper_baseline, lower_baseline,96*15,96*16)
# 200 201 非常ok
# 

## 方案2结果分析

In [17]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates_10%_1h/range_width_1'
stata_folder = 'ch5_data/analyse_results_of_dates_10%_1h/range_width_1'

stata_list_2,anly_list_2 = statistics(data_anly_2,refine_foler,stata_folder)
for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_2[i]}')

减少的考核电量是506395.1734226183
原考核电量是1107604.9910515868
后考核电量是601209.8176289685
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是5264
优化后向上波动考核点数量是2023
优化后向下波动考核点数量是3241
原受考核点比例是0.25408830160678103
原向上波动考核点比例是0.1274579753988413
原向下波动考核点比例是0.12663032620793974
优化后受考核点比例是0.15023259796227062
优化后向上波动考核点比例是0.0577356659722024
优化后向下波动考核点比例是0.09249693199006821
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是241556.83000000007
后向上波动考核功率求和是-403159.15


In [18]:
visualization_interactive(data_anly_2, upper_baseline, lower_baseline,96*15,96*16)


## 方案3结果分析

In [19]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates_10%_1h/range_width_30'
stata_folder = 'ch5_data/analyse_results_of_dates_10%_1h/range_width_30'

stata_list_3,anly_list_3 = statistics(data_anly_3,refine_foler,stata_folder)
for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_3[i]}')

减少的考核电量是869153.961051587
原考核电量是1107604.9910515868
后考核电量是238451.0299999998
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是3104
优化后向上波动考核点数量是1566
优化后向下波动考核点数量是1538
原受考核点比例是0.25429877177949156
原向上波动考核点比例是0.12756355327049415
原向下波动考核点比例是0.12673521850899744
优化后受考核点比例是0.08866038274778634
优化后向上波动考核点比例是0.04473007712082262
优化后向下波动考核点比例是0.04393030562696373
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是132741.27000000014
后向上波动考核功率求和是-127076.51000000013


In [20]:
visualization_interactive(data_anly_3, upper_baseline, lower_baseline,96*15,96*16)


## 方案4结果分析

In [21]:
# 一个标准的调用函数示例
# 输入一个数据集，

refine_foler = 'ch5_data/analyse_results_of_dates_10%_1h/range_with_halfE'
stata_folder = 'ch5_data/analyse_results_of_dates_10%_1h/range_with_halfE'

stata_list_4,anly_list_4 = statistics(data_anly_4,refine_foler,stata_folder)
for i in range(len(print_list)):
    print(f'{print_list[i]}是{anly_list_4[i]}')

减少的考核电量是778021.2135515867
原考核电量是1107604.9910515868
后考核电量是329583.7775000001
原受考核点数量是8903
原向上波动考核点数量是4466
原向下波动考核点数量是4437
优化后受考核点数量是3232
优化后向上波动考核点数量是1637
优化后向下波动考核点数量是1595
原受考核点比例是0.25429877177949156
原向上波动考核点比例是0.12756355327049415
原向下波动考核点比例是0.12673521850899744
优化后受考核点比例是0.09231648100542703
优化后向上波动考核点比例是0.04675806912310768
优化后向下波动考核点比例是0.045558411882319336
原向上波动考核功率求和是535053.67
原向上波动考核功率求和是-520099.51
后向上波动考核功率求和是154609.53000000014
后向上波动考核功率求和是-147373.80000000016


In [22]:
visualization_interactive(data_anly_4, upper_baseline, lower_baseline,96*15,96*16)

In [23]:
anly_result_df = pd.DataFrame({
        '方案1':np.ceil(anly_list_1),
        '方案2': np.ceil(anly_list_2),
        '方案3': np.ceil(anly_list_3),
        '方案4':np.ceil(anly_list_4)
    })

In [24]:
anly_result_df_T = anly_result_df.T
anly_result_df_T.columns = ['减少的考核电量',
                          '原考核电量',
                          '后考核电量',
                          '原受考核点数量',
                          '原向上波动考核点数量',
                          '原向下波动考核点数量',
                          '优化后受考核点数量',
                          '优化后向上波动考核点数量',
                          '优化后向下波动考核点数量',
                          '原受考核点比例',
                          '原向上波动考核点比例',
                          '原向下波动考核点比例',
                          '优化后受考核点比例',
                          '优化后向上波动考核点比例',
                          '优化后向下波动考核点比例',
                          '原向上波动考核功率求和',
                          '原向下波动考核功率求和',
                          '后向上波动考核功率求和',
                          '后向下波动考核功率求和']

In [25]:
anly_result_df_T.T.to_csv('ch5_data/analyse_results_of_dates_10%_1h/各方案运行结果总体分析.csv')
anly_result_df_T['优化的电量的比例'] = np.round(anly_result_df_T['减少的考核电量']/anly_result_df_T['原考核电量'],2)
anly_result_df_T.T

,方案1,方案2,方案3,方案4
减少的考核电量,836949.00,506396.00,869154.00,778022.0
原考核电量,1107605.00,1107605.00,1107605.00,1107605.0
后考核电量,270657.00,601210.00,238452.00,329584.0
原受考核点数量,8903.00,8903.00,8903.00,8903.0
原向上波动考核点数量,4466.00,4466.00,4466.00,4466.0
原向下波动考核点数量,4437.00,4437.00,4437.00,4437.0
优化后受考核点数量,3514.00,5264.00,3104.00,3232.0
优化后向上波动考核点数量,1713.00,2023.00,1566.00,1637.0
优化后向下波动考核点数量,1801.00,3241.00,1538.00,1595.0
原受考核点比例,1.00,1.00,1.00,1.0


# 连续波动分析

In [26]:
def continuou_fluctuation_analysis(df,mode):
    # 功能描述：分析连续波动的各项指标
    # 函数输入：原始数据集，模式，保存文件夹
    # 函数输出：分析数据集
    df_positive_copy = df.copy()

    if mode == 'positive':
        df_positive_copy['is_positive'] = df_positive_copy['after_flucuation'] > 0
        df_positive_copy['is_positive_above'] = df_positive_copy['after_flucuation'] > 50
    elif mode == 'negative':
        df_positive_copy['is_positive'] = df_positive_copy['after_flucuation'] < 0
        df_positive_copy['is_positive_above'] = df_positive_copy['after_flucuation'] < -50

    # # 进行绝对值的处理
    # df_positive_copy['is_positive'] = abs(df_positive_copy['is_positive'])
    df_positive_copy['after_flucuation'] = abs(df_positive_copy['after_flucuation'])
    
    # 检测连续正数序列的开始
    df_positive_copy['new_group'] = df_positive_copy['is_positive_above'] & (df_positive_copy['is_positive_above'] != df_positive_copy['is_positive_above'].shift(1))
    # 为每个连续正数序列分配一个唯一的标识
    df_positive_copy['group_id'] = df_positive_copy['new_group'].cumsum()
    # 仅选取正数进行分组和计数
    positive_sequences = df_positive_copy[df_positive_copy['is_positive_above']].groupby('group_id').size()
    # 首先对每个连续正数序列进行聚合计算，得到波动总值、平均值和最大值
    grouped_stats = df_positive_copy[df_positive_copy['is_positive_above']].groupby('group_id')['after_flucuation'].agg(
        total_fluctuation='sum',
        average_fluctuation='mean',
        max_fluctuation='max'
    ).reset_index()

    # 将原先计算的序列长度也加入统计表中
    grouped_stats['sequence_length'] = grouped_stats['group_id'].map(positive_sequences)

    # 现在grouped_stats包含了每个连续正数序列的长度、波动总值、平均值和最大值
    # 对这些统计值进行进一步的汇总统计
    summary_stats = grouped_stats.groupby('sequence_length').agg({
        'total_fluctuation': ['sum', 'mean', 'max'],
        'average_fluctuation': ['mean'],
        'max_fluctuation': ['max']
    })
    # 计算各长度连续波动的数量
    sequence_counts = positive_sequences.groupby(positive_sequences).size().reset_index()
    sequence_counts.columns = ['sequence_length', 'count']

    # 合并各项统计数据
    summary_stats = pd.merge(summary_stats, sequence_counts, on='sequence_length', how='left')

    # 重命名列以便更清晰地表示统计结果
    summary_stats.columns = ['sequence_length', 'total_fluctuation_sum', 'total_fluctuation_mean', 'total_fluctuation_max',
                             'average_fluctuation_mean', 'max_fluctuation_max', 'count']
    # # 重命名列以便更清晰地表示统计结果
    # summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns.values]
    # summary_stats = summary_stats.reset_index()

    return summary_stats



In [27]:
def method_continuou_fluctuation_analysis(df):
    df_positive_copy = df.copy()
    df_negative_copy = df.copy()
    positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
    positive_con_sum = positive_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向上越限波动段功率差的和值的总和
    positive_con_max = positive_summary_stats['total_fluctuation_max'][1:].max() # 连续向上波动段和值功率差的和值的最大值
    positive_con_front_max = positive_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向上波动段和值功率差的和值的最大值
    
    negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
    negative_con_sum = negative_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向下越限波动段功率差的和值的总和
    negative_con_max = negative_summary_stats['total_fluctuation_max'][1:].max() # 连续向下波动段和值功率差的和值的最大值
    negative_con_front_max = negative_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向下波动段和值功率差的和值的最大值
    result_list = [positive_con_sum,positive_con_max,positive_con_front_max,negative_con_sum,negative_con_max,negative_con_front_max]

    return result_list

## 方案1

### 正向波动

In [28]:
df_positive_copy = data_anly_1.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案1的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,92246.98,128.298999,1166.16,128.298999,1166.16,719
1,2,71749.53,279.181051,1054.79,139.590525,743.67,257
2,3,33686.04,461.452603,1469.26,153.817534,658.09,73
3,4,17134.24,611.937143,1168.71,152.984286,451.15,28
4,5,7922.30,792.230000,1947.40,158.446000,850.13,10
5,6,7901.43,1128.775714,1792.46,188.129286,981.36,7
6,7,3971.69,992.922500,1376.13,141.846071,355.33,4
7,9,1353.53,1353.530000,1353.53,150.392222,236.59,1
8,10,2659.74,1329.870000,1550.82,132.987000,274.58,2


### 反向波动

In [29]:
df_negative_copy = data_anly_1.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案1的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,72522.74,110.891040,788.40,110.891040,788.40,654
1,2,58649.19,245.394100,1330.52,122.697050,1033.96,239
2,3,38032.69,404.603085,1136.98,134.867695,561.04,94
3,4,22191.50,583.986842,1769.89,145.996711,1218.68,38
4,5,9737.08,749.006154,1373.95,149.801231,572.58,13
5,6,10391.49,799.345385,1625.33,133.224231,541.12,13
6,7,6009.69,858.527143,1360.30,122.646735,403.86,7
7,8,2278.56,1139.280000,1307.79,142.410000,502.98,2
8,9,3209.07,1069.690000,1590.78,118.854444,439.46,3


## 方案2

### 正向波动

In [30]:
df_positive_copy = data_anly_2.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案2的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,53251.31,67.577805,1145.58,67.577805,1145.58,788
1,2,44263.19,258.849064,884.64,129.424532,802.48,171
2,3,37234.49,443.267738,1510.95,147.755913,816.66,84
3,4,41993.81,646.058615,1193.18,161.514654,849.67,65
4,5,35607.61,913.015641,1849.27,182.603128,772.14,39
5,6,13188.04,942.002857,1535.24,157.000476,692.27,14
6,7,9067.41,1133.426250,1344.82,161.918036,387.61,8
7,8,3828.92,1276.306667,2001.31,159.538333,518.87,3
8,9,1386.74,1386.740000,1386.74,154.082222,232.50,1
9,13,1735.31,1735.310000,1735.31,133.485385,319.30,1


In [31]:
df_negative_copy = data_anly_2.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案2的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,84147.62,101.750447,691.46,101.750447,691.46,827
1,2,119511.28,244.399346,1088.97,122.199673,871.08,489
2,3,75040.25,414.587017,1176.60,138.195672,942.31,181
3,4,61572.01,615.720100,1784.21,153.930025,812.01,100
4,5,19108.63,682.451071,1680.31,136.490214,572.58,28
5,6,16208.04,736.729091,1331.87,122.788182,390.00,22
6,7,9518.72,865.338182,1410.31,123.619740,373.69,11
7,8,3030.92,1010.306667,1249.19,126.288333,280.50,3
8,9,4343.78,1085.945000,1206.82,120.660556,269.49,4
9,10,1579.47,1579.470000,1579.47,157.947000,310.79,1


## 方案3

### 正向波动

In [32]:
df_positive_copy = data_anly_3.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案3的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,53300.65,59.487333,923.44,59.487333,923.44,896
1,2,17966.10,179.661000,876.05,89.830500,712.97,100
2,3,23878.63,397.977167,1333.21,132.659056,1106.66,60
3,4,16387.48,565.085517,1310.49,141.271379,560.10,29
4,5,10134.57,596.151176,885.54,119.230235,391.91,17
5,6,3533.69,883.422500,1576.30,147.237083,820.51,4
6,7,4379.41,875.882000,1125.17,125.126000,270.89,5
7,9,1844.11,922.055000,1237.96,102.450556,455.59,2
8,12,1316.63,1316.630000,1316.63,109.719167,226.79,1


### 反向波动

In [33]:
df_negative_copy = data_anly_3.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案3的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,55220.91,57.402193,691.46,57.402193,691.46,962
1,2,26187.71,231.749646,1030.12,115.874823,681.95,113
2,3,16901.56,402.418095,1353.48,134.139365,767.80,42
3,4,10776.77,513.179524,1235.11,128.294881,496.36,21
4,5,6177.33,772.166250,1266.58,154.433250,500.73,8
5,6,6555.07,728.341111,1339.70,121.390185,518.41,9
6,7,2633.16,877.720000,1130.36,125.388571,367.57,3
7,8,1787.13,893.565000,1044.62,111.695625,278.98,2
8,9,836.87,836.870000,836.87,92.985556,174.93,1


## 方案4

### 正向波动

In [34]:
df_positive_copy = data_anly_4.copy()
positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
positive_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案4的连续向上越限波动统计.csv')
positive_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,60500.25,59.841988,1014.27,59.841988,1014.27,1011
1,2,32963.87,302.420826,903.66,151.210413,681.70,109
2,3,31503.80,470.205970,1339.17,156.735323,1106.62,67
3,4,17092.40,633.051852,1656.56,158.262963,741.29,27
4,5,2130.59,532.647500,909.21,106.529500,322.60,4
5,6,4566.28,913.256000,1139.65,152.209333,548.90,5
6,7,2967.78,741.945000,961.89,105.992143,290.73,4
7,10,1413.39,1413.390000,1413.39,141.339000,423.72,1
8,11,1471.17,1471.170000,1471.17,133.742727,260.32,1


### 反向波动

In [35]:
df_negative_copy = data_anly_4.copy()
negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
negative_summary_stats.to_csv('ch5_data/analyse_results_of_dates_10%_1h/方案4的连续向下越限波动统计.csv')
negative_summary_stats

,sequence_length,total_fluctuation_sum,total_fluctuation_mean,total_fluctuation_max,average_fluctuation_mean,max_fluctuation_max,count
0,1,56641.29,60.970172,691.46,60.970172,691.46,929
1,2,36932.17,269.577883,1205.17,134.788942,826.33,137
2,3,22095.79,450.934490,1565.92,150.311497,983.33,49
3,4,8900.88,523.581176,1328.57,130.895294,610.02,17
4,5,5400.12,675.015000,1312.46,135.003000,551.38,8
5,6,5160.86,737.265714,1211.75,122.877619,384.25,7
6,7,3985.71,996.427500,1255.31,142.346786,468.47,4
7,8,4539.93,1134.982500,1294.74,141.872813,350.64,4
8,11,2296.74,1148.370000,1229.51,104.397273,211.81,2
9,13,1420.31,1420.310000,1420.31,109.254615,204.85,1


In [36]:
def method_continuou_fluctuation_analysis(df):
    df_positive_copy = df.copy()
    df_negative_copy = df.copy()
    positive_summary_stats = continuou_fluctuation_analysis(df_positive_copy,'positive')
    positive_con_sum = positive_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向上越限波动段功率差的和值的总和
    positive_con_mean = positive_summary_stats['total_fluctuation_sum'][1:].sum()/positive_summary_stats['count'][1:].sum()
    positive_con_max = positive_summary_stats['total_fluctuation_max'][1:].max() # 连续向上波动段和值功率差的和值的最大值
    positive_con_front_max = positive_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向上波动段和值功率差的和值的最大值
    positive_count = positive_summary_stats['count'][1:].sum()
    positive_front_count = positive_summary_stats['count'][1:5].sum()

    negative_summary_stats = continuou_fluctuation_analysis(df_negative_copy,'negative')
    negative_con_sum = negative_summary_stats['total_fluctuation_sum'][1:].sum() # 连续向下越限波动段功率差的和值的总和
    negative_con_mean = negative_summary_stats['total_fluctuation_sum'][1:].sum()/positive_summary_stats['count'][1:].sum()
    negative_con_max = negative_summary_stats['total_fluctuation_max'][1:].max() # 连续向下波动段和值功率差的和值的最大值
    negative_con_front_max = negative_summary_stats['total_fluctuation_max'][1:5].max() # 长度为2~5的连续向下波动段和值功率差的和值的最大值
    negative_count = positive_summary_stats['count'][1:].sum()
    negative_front_count = positive_summary_stats['count'][1:5].sum()

    result_list = [positive_con_sum,
                   round(positive_con_mean,2),
                   positive_con_max,
                   positive_con_front_max,
                   positive_count,
                   positive_front_count,
                   negative_con_sum,
                   round(negative_con_mean,2),
                   negative_con_max,
                   negative_con_front_max,
                   negative_count,
                   negative_front_count]

    return result_list

In [37]:
# 指标统计
method_1_result_list = method_continuou_fluctuation_analysis(data_anly_1)
method_2_result_list = method_continuou_fluctuation_analysis(data_anly_2)
method_3_result_list = method_continuou_fluctuation_analysis(data_anly_3)
method_4_result_list = method_continuou_fluctuation_analysis(data_anly_4)

data_original = data_anly_1.copy()
data_original['after_flucuation'] = data_original['original_flucuation']
original_result_list = method_continuou_fluctuation_analysis(data_original)
original_result_list

[411213.3099999999,
 364.87,
 2001.29,
 1834.0900000000001,
 1127,
 1066,
 388487.55,
 344.71,
 1834.2,
 1834.2,
 1127,
 1066]

In [38]:
method_con_result_df = pd.DataFrame({
        '无储能':original_result_list,
        '方案1':method_1_result_list,
        '方案2': method_2_result_list,
        '方案3': method_3_result_list,
        '方案4':method_4_result_list
    })

In [39]:
method_con_result_df_T = method_con_result_df.T
method_con_result_df_T.columns = ['连续向上越限波动段功率差的和值的总和',
                                  '连续向上波动段功率差和值的平均值',
                                  '连续向上波动段和值功率差的和值的最大值',
                                  '长度为2~5的连续向上波动段和值功率差的和值的最大值',
                                  '连续向上波动段计数',
                                  '长度为2~5的连续向上波动段计数',
                                  '连续向下越限波动段功率差的和值的总和',
                                  '连续向下波动段功率差和值的平均值',
                                  '连续向下波动段和值功率差的和值的最大值',
                                  '长度为2~5的连续向下波动段和值功率差的和值的最大值',
                                  '连续向下波动段计数',
                                  '长度为2~5的连续向下波动段计数']
method_con_result_df_T.to_csv("ch5_data/analyse_results_of_dates_10%_1h/不同方案的连续越限波动统计.csv")
method_con_result_df_T

,连续向上越限波动段功率差的和值的总和,连续向上波动段功率差和值的平均值,连续向上波动段和值功率差的和值的最大值,长度为2~5的连续向上波动段和值功率差的和值的最大值,连续向上波动段计数,长度为2~5的连续向上波动段计数,连续向下越限波动段功率差的和值的总和,连续向下波动段功率差和值的平均值,连续向下波动段和值功率差的和值的最大值,长度为2~5的连续向下波动段和值功率差的和值的最大值,连续向下波动段计数,长度为2~5的连续向下波动段计数
无储能,411213.31,364.87,2001.29,1834.09,1127.0,1066.0,388487.55,344.71,1834.20,1834.20,1127.0,1066.0
方案1,146378.50,383.19,1947.40,1947.40,382.0,368.0,150499.27,393.98,1769.89,1769.89,382.0,368.0
方案2,188305.52,487.84,2001.31,1849.27,386.0,359.0,319011.53,826.45,1904.33,1784.21,386.0,359.0
方案3,79440.62,364.41,1576.30,1333.21,218.0,206.0,71855.60,329.61,1353.48,1353.48,218.0,206.0
方案4,94109.28,431.69,1656.56,1656.56,218.0,207.0,90732.51,416.20,1565.92,1565.92,218.0,207.0


In [40]:
method_con_result_df_T = method_con_result_df_T.reset_index()
method_con_result_df_T

,index,连续向上越限波动段功率差的和值的总和,连续向上波动段功率差和值的平均值,连续向上波动段和值功率差的和值的最大值,长度为2~5的连续向上波动段和值功率差的和值的最大值,连续向上波动段计数,长度为2~5的连续向上波动段计数,连续向下越限波动段功率差的和值的总和,连续向下波动段功率差和值的平均值,连续向下波动段和值功率差的和值的最大值,长度为2~5的连续向下波动段和值功率差的和值的最大值,连续向下波动段计数,长度为2~5的连续向下波动段计数
0,无储能,411213.31,364.87,2001.29,1834.09,1127.0,1066.0,388487.55,344.71,1834.20,1834.20,1127.0,1066.0
1,方案1,146378.50,383.19,1947.40,1947.40,382.0,368.0,150499.27,393.98,1769.89,1769.89,382.0,368.0
2,方案2,188305.52,487.84,2001.31,1849.27,386.0,359.0,319011.53,826.45,1904.33,1784.21,386.0,359.0
3,方案3,79440.62,364.41,1576.30,1333.21,218.0,206.0,71855.60,329.61,1353.48,1353.48,218.0,206.0
4,方案4,94109.28,431.69,1656.56,1656.56,218.0,207.0,90732.51,416.20,1565.92,1565.92,218.0,207.0


## 连续波动画图

In [41]:

fig = px.bar(
    x=method_con_result_df_T['index'],
    y=["连续向上越限波动段功率差的和值的总和", '连续向下越限波动段功率差的和值的总和'],
    labels={'index': '方案', 'value': '功率值 (MW)', 'variable':'分析指标'},
    data_frame=method_con_result_df_T,
    barmode='group',
)
# 更新图表的布局设置，包括字体
fig.update_layout(
    xaxis_title='方案',
    yaxis_title='功率值 (MW)',
    yaxis_title_standoff=25,
    font=dict(family="SimHei", size=20)
)




In [42]:

fig = px.bar(
    x=method_con_result_df_T['index'],
    y= ["连续向上波动段和值功率差的和值的最大值",
        '连续向下波动段和值功率差的和值的最大值',
        '长度为2~5的连续向上波动段和值功率差的和值的最大值',
        '长度为2~5的连续向下波动段和值功率差的和值的最大值',
        '连续向上波动段功率差和值的平均值',
        '连续向下波动段功率差和值的平均值'
        ],  
    data_frame=method_con_result_df_T,
    labels={'index': '方案', 'value': '功率值 (MW)', 'variable':'分析指标'},
    barmode='group',
)
fig.update_layout(
    xaxis_title='方案',
    yaxis_title='功率值 (MW)',
    yaxis_title_standoff=25,
    font=dict(family="SimHei", size=20)
)
fig.show() 

In [43]:
fig = px.bar(
    x=method_con_result_df_T['index'],
    y= ["连续向上波动段计数",
        '连续向下波动段计数',
        '长度为2~5的连续向上波动段计数',
        '长度为2~5的连续向下波动段计数'
        ],  
    data_frame=method_con_result_df_T,
    labels={'index': '方案', 'value': '段数', 'variable':'分析指标'},
    barmode='group',
)
fig.update_layout(
    xaxis_title='方案',
    yaxis_title='段数',
    yaxis_title_standoff=25,
    font=dict(family="SimHei", size=20)
)
fig.show() 